In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Bu bölümde, projede kullanacağımız temel verileri `.csv` dosyalarından yüklüyoruz:
- results.csv → yarış sonuçları
- races.csv → yarış bilgileri (tarih, yıl vs.)
- drivers.csv → sürücü bilgileri
- constructors.csv → takım bilgileri

In [39]:
df_results = pd.read_csv("dataset/results.csv")
df_races = pd.read_csv("dataset/races.csv")
df_drivers = pd.read_csv("dataset/drivers.csv")
df_constructors = pd.read_csv("dataset/constructors.csv")

Hedef Değişkenin Oluşturulması:
Makine öğrenmesi modelimde tahmin etmeye çalışacağımız hedef, bir pilotun yarışı ilk 3 sırada tamamlayıp tamamlamadığı.
`results.csv` dosyasındaki `positionOrder` değişkeni kullanılarak, `top3` adında ikili (binary) bir sınıflandırma etiketi oluşturuyoruz:
   Eğer `positionOrder <= 3` ise: 1
   Aksi halde: 0

In [ ]:
# Yarışta ilk 3'e girip girmediğini belirten yeni sütun
df_results['top3'] = df_results['positionOrder'].apply(lambda x: 1 if x <= 3 else 0)

# Örnek ilk 5 veri
df_results[['driverId', 'positionOrder', 'top3']].head()

Veri Birleştirme (Merging):
Modelde kullanacağımız öznitelikleri (feature) oluşturmak için, yarış sonuçları ile birlikte:
- yarış bilgilerini (yarış yılı gibi),
- pilot bilgilerini (milliyet gibi)
- takım bilgilerini (takım ismi)
birleştirerek tek bir tablo haline getireceğiz.

In [ ]:
# results + races → yarış yılı gibi bilgiler
df = df_results.merge(df_races[['raceId', 'year', 'circuitId']], on='raceId', how='left')

# results + drivers → milliyet gibi bilgiler
df = df.merge(df_drivers[['driverId', 'nationality']], on='driverId', how='left')

# results + constructors → takım ismi gibi bilgiler
df = df.merge(df_constructors[['constructorId', 'name']], on='constructorId', how='left')

# İlk 5 satıra bakalım
df.head()

Eksik Verilerin Kontrolü (Özel NaN Tanımıyla):
 Veri setinde bazı eksik değerler doğrudan `NaN` yerine `"\\N"` ya da boş string (`""`) olarak kaydedilmiş olabilir.  
 Bu nedenle `na_values=["\\N", ""]` argümanıyla CSV dosyalarını tekrar okuduk ve şimdi eksik verileri daha doğru şekilde analiz edebiliyoruz.

In [26]:
# Eksik veri değerlerini algılayabilmesi için özel ayarlarla tekrar okuyoruz
na_values = ["\\N", ""]  # Boşluk ve '\N' stringleri artık NaN sayılacak

df_results = pd.read_csv("results.csv", na_values=na_values)
df_races = pd.read_csv("races.csv", na_values=na_values)
df_drivers = pd.read_csv("drivers.csv", na_values=na_values)
df_constructors = pd.read_csv("constructors.csv", na_values=na_values)

# Hedef değişkeni tekrar oluştur
df_results['top3'] = df_results['positionOrder'].apply(lambda x: 1 if x <= 3 else 0)

# Verileri yeniden birleştir
df = df_results.merge(df_races[['raceId', 'year', 'circuitId']], on='raceId', how='left')
df = df.merge(df_drivers[['driverId', 'nationality']], on='driverId', how='left')
df = df.merge(df_constructors[['constructorId', 'name']], on='constructorId', how='left')


In [ ]:
df.isnull().sum().sort_values(ascending=False)

Eksik Verilerin Temizlenmesi:
 Model eğitimi için gerekli olmayan, çok sayıda eksik veri içeren sütunları veri setimizden çıkartıyoruz.  
 Az sayıda eksik değere sahip sütunlarda ise, eksik değerleri uygun şekilde dolduruyoruz.

In [29]:
df.columns = df.columns.str.strip()

In [ ]:
cols_to_drop = ['milliseconds', 'time', 'fastestLap', 'fastestLapTime', 'fastestLapSpeed', 'rank', 'position']
existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
df.drop(columns=existing_cols_to_drop, inplace=True)

In [33]:
df.dropna(subset=['number'], inplace=True)

In [ ]:
df.isnull().sum().sort_values(ascending=False)

Kategorik Verileri Sayısal Değerlere Çevirme (Label Encoding)

Makine öğrenmesi modelleri, metin (kategorik) verileri doğrudan kullanamazlar. Bu nedenle, `nationality` (milliyet) ve `name` (takım adı) gibi kategorik sütunları sayısal değerlere dönüştürmemiz gerekir.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Label Encoder nesnelerini oluşturuyoruz
le_nationality = LabelEncoder()
le_name = LabelEncoder()

# 'nationality' sütununu sayısal hale çeviriyoruz
df['nationality_enc'] = le_nationality.fit_transform(df['nationality'])

# 'name' sütununu sayısal hale çeviriyoruz
df['name_enc'] = le_name.fit_transform(df['name'])

df[['nationality', 'nationality_enc', 'name', 'name_enc']].head()

Temizlenmiş ve ön işleme tabi tutulmuş veri setimizi pickle formatında kaydederek, ileride kolayca yükleyip kullanabiliriz.

In [38]:
# Temizlenmiş veri setini pickle formatında kaydet
df.to_pickle("f1_cleaned_data.pkl")